<a href="https://colab.research.google.com/github/heyronith/heyronith/blob/main/Final_Fungal_Infection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import os

# Mount your Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the parameters
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32
DIRECTORY = '/content/drive/MyDrive/C'

# Apply data augmentation
datagen = ImageDataGenerator(rescale=1.0/255.0,
                             rotation_range=20,
                             zoom_range=0.15,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.15,
                             horizontal_flip=True,
                             fill_mode="nearest",
                             validation_split=0.2)

# Prepare the iterators
train_it = datagen.flow_from_directory(DIRECTORY,
                                       target_size=IMAGE_SIZE,
                                       batch_size=BATCH_SIZE,
                                       class_mode='categorical',
                                       color_mode='grayscale',  # Set color_mode to 'grayscale'
                                       subset='training')
val_it = datagen.flow_from_directory(DIRECTORY,
                                     target_size=IMAGE_SIZE,
                                     batch_size=BATCH_SIZE,
                                     class_mode='categorical',
                                     color_mode='grayscale',  # Set color_mode to 'grayscale'
                                     subset='validation')

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=IMAGE_SIZE + (1,)))  # Change the number of channels to 1
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_it.num_classes, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


history = model.fit(train_it,
                    steps_per_epoch=len(train_it),
                    validation_data=val_it,
                    validation_steps=len(val_it),
                    epochs=25)




Mounted at /content/drive
Found 1680 images belonging to 4 classes.
Found 417 images belonging to 4 classes.
Epoch 1/25
53/53 [==============================] - 10s 161ms/step - loss: 1.1403 - accuracy: 0.4821 - val_loss: 0.8147 - val_accuracy: 0.6499
Epoch 2/25
53/53 [==============================] - 7s 138ms/step - loss: 0.7679 - accuracy: 0.6381 - val_loss: 0.6893 - val_accuracy: 0.6715
Epoch 3/25
53/53 [==============================] - 8s 148ms/step - loss: 0.6780 - accuracy: 0.6893 - val_loss: 0.6354 - val_accuracy: 0.6882
Epoch 4/25
53/53 [==============================] - 7s 131ms/step - loss: 0.6598 - accuracy: 0.6851 - val_loss: 0.6496 - val_accuracy: 0.7026
Epoch 5/25
53/53 [==============================] - 8s 155ms/step - loss: 0.6065 - accuracy: 0.7238 - val_loss: 0.6450 - val_accuracy: 0.6715
Epoch 6/25
53/53 [==============================] - 8s 142ms/step - loss: 0.6284 - accuracy: 0.7083 - val_loss: 0.6034 - val_accuracy: 0.7458
Epoch 7/25
53/53 [====================

In [ ]:
model.save('model_cnn.h5')

In [ ]:
loss, accuracy = model.evaluate(val_it, steps=len(val_it))
print("Validation accuracy: {:.2f}%".format(accuracy * 100))

14/14 [==============================] - 2s 103ms/step - loss: 0.5659 - accuracy: 0.7386
Validation accuracy: 73.86%


In [ ]:
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import img_to_array  # Correct import here

def load_image(img_path, size):
    # Load the image
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Convert the image to 3 channels
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    # Resize the image
    img = cv2.resize(img, size)

    # Convert the image data to a 4D tensor
    img_tensor = img_to_array(img)  # Correct usage here

    # The model expects input in this form ((1, size, size, 3))
    img_tensor = np.expand_dims(img_tensor, axis=0)

    # Follow the Original Model preprocessing
    img_tensor /= 255.

    return img_tensor

img_path = "/content/OIP.jpg"

# The original model was trained on 64x64 images
img = load_image(img_path, size=(64,64))


# Get the index of the highest probability
class_index = np.argmax(history)

# Now get the class label
class_label = train_it.class_indices
class_label = {v: k for k, v in class_label.items()}
predicted_class = class_label[class_index]

print(predicted_class)


Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
